In [5]:
import atlite
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import os
import numpy as np

In [2]:


# Folder containing the input NetCDF files
folder = "Cutouts_atlite"

# List of input NetCDF files in the folder
input_files = [os.path.join(folder, f"Laos{i}Y.nc") for i in range(1, 6)]

# Load all files as xarray datasets and ensure they align
datasets = [xr.open_dataset(file) for file in input_files]

# Calculate the average across all datasets
average_dataset = xr.concat(datasets, dim="time").mean(dim="time")

# Save the averaged dataset to a new NetCDF file
output_file = os.path.join(folder, "Laos5AVG.nc")
average_dataset.to_netcdf(output_file)

# Close all input datasets
for ds in datasets:
    ds.close()

print(f"Averaged dataset saved to '{output_file}'")


Averaged dataset saved to 'Cutouts_atlite\Laos5AVG.nc'


### Plot

In [12]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts

    Parameters
    ----------
    eta : float
        Efficiency of the hydropower plant.
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.

    Returns
    -------
    float
        Hydropower potential in Megawatts (MW).
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    '''
    Calculate the hydropower potential considering the capacity limit

    Parameters
    ----------
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.
    capacity : float
        Maximum hydropower capacity in Megawatts (MW).
    eta : float
        Efficiency of the hydropower plant.

    Returns
    -------
    xarray DataArray
        Capacity factor, which is the limited potential divided by the capacity.
    '''
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

In [ ]:
## Have to create capacity factors

In [13]:
datasets

[<xarray.Dataset> Size: 590MB
 Dimensions:               (x: 30, y: 35, time: 8784)
 Coordinates:
   * x                     (x) float64 240B 100.2 100.5 100.8 ... 107.2 107.5
   * y                     (y) float64 280B 14.0 14.25 14.5 ... 22.0 22.25 22.5
   * time                  (time) datetime64[ns] 70kB 2019-01-01 ... 2020-01-0...
     lon                   (x) float64 240B ...
     lat                   (y) float64 280B ...
 Data variables: (12/15)
     height                (y, x) float32 4kB ...
     wnd100m               (time, y, x) float32 37MB ...
     wnd_shear_exp         (time, y, x) float32 37MB ...
     wnd_azimuth           (time, y, x) float32 37MB ...
     roughness             (time, y, x) float32 37MB ...
     influx_toa            (time, y, x) float32 37MB ...
     ...                    ...
     solar_altitude        (time, y, x) float64 74MB ...
     solar_azimuth         (time, y, x) float64 74MB ...
     temperature           (time, y, x) float32 37MB ...
   

### Create one plot of the average capacity factor over each year, including actual values